#Imports

In [1]:
#@title
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
import numpy as np
!git clone https://github.com/bosemessi/StatsbombOpenData --q
!pip install mplsoccer --q
from mplsoccer import Pitch, VerticalPitch
import ipywidgets as widgets
from ipywidgets import interact,interactive
import matplotlib.pyplot as plt
import matplotlib as mpl 
import matplotlib.font_manager as fm
!pip install highlight-text --q
from highlight_text import ax_text, fig_text
!pip install adjustText --q
from adjustText import adjust_text
!pip install gdown --quiet
import gdown
import matplotlib.patheffects as path_effects
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes
import mpl_toolkits.axisartist.angle_helper as angle_helper
from matplotlib.projections import PolarAxes
from mpl_toolkits.axisartist.grid_finder import (FixedLocator, MaxNLocator,
                                                 DictFormatter)
import matplotlib.patches as patches
from PIL import Image
from io import StringIO, BytesIO
from tqdm import tqdm
import requests 
import warnings
warnings.filterwarnings('ignore')
from matplotlib.colors import LinearSegmentedColormap
from scipy.interpolate import RegularGridInterpolator

     |████████████████████████████████| 61kB 2.2MB/s 


In [2]:
#@title
df = pd.read_parquet('/content/StatsbombOpenData/WC2018.parquet').reset_index(drop=True)

def player_minutes(match_id, team_name):
    gamedf = df[(df.match_id==match_id)&(df.period<=4)].reset_index(drop=True)
    team1 = gamedf.team_name[0]
    team2 = gamedf.team_name[1]
    if team_name == team1:
        lineup = [gamedf.tactics_lineup[0][i]['player']['name'] for i in range(11)]
    else:
        lineup = [gamedf.tactics_lineup[1][i]['player']['name'] for i in range(11)]
    allplayers = gamedf[(gamedf.team_name==team_name)&
                        (gamedf.player_name.notna())].player_name.unique().tolist()
    
    starters = [name for name in allplayers if name in lineup]
    playerons = [name for name in allplayers if name not in lineup]
    playeroffs = [gamedf[gamedf.substitution_replacement_name==name].player_name.values 
                  for name in playerons]
    rednames = gamedf[gamedf['foul_committed_card_name'].isin(['Second Yellow',
                                                            'Red Card'])].player_name.unique().tolist()
    redtimes = gamedf[gamedf['foul_committed_card_name'].isin(['Second Yellow',
                                                            'Red Card'])].minute.unique().tolist()
    timeoffs = [gamedf[gamedf.substitution_replacement_name==name].minute.values.item() 
                                for name in playerons]
    playeroffs = playeroffs + rednames
    timeoffs = timeoffs + redtimes
    timeons = [gamedf[gamedf.substitution_replacement_name==name].minute.values.item() 
                                for name in playerons]
    max_min = gamedf.minute.max()
    time_played = []

    for i, pl in enumerate(allplayers):
        if((pl in playerons) & (pl not in playeroffs)):
            time_played.append(max_min - timeons[playerons.index(pl)]) 
        elif((pl in playerons) & (pl in playeroffs)):
            time_played.append(timeoffs[playeroffs.index(pl)] - timeons[playerons.index(pl)])
        elif(pl in playeroffs):
            time_played.append(timeoffs[playeroffs.index(pl)])
        else:
            time_played.append(max_min)

    smalltimedf = pd.DataFrame()
    smalltimedf['name'] = allplayers
    smalltimedf['time'] = time_played
    return smalltimedf
    
dfs = []
for m in tqdm(df.match_id.unique().tolist(),desc='Looping through matches'):
    teams = df[df.match_id==m].team_name.unique()
    print(teams[0], teams[1], m) 
    sdf1 = player_minutes(m, teams[0])
    sdf2 = player_minutes(m, teams[1])
    dfs.append(sdf1)
    dfs.append(sdf2)
mastertimedf = pd.concat(dfs, ignore_index=True)
groupedtime = mastertimedf.groupby(['name']).agg({'time':'sum'}).reset_index()

Looping through matches:   3%|▎         | 2/64 [00:00<00:04, 15.37it/s]

Croatia Denmark 7581
Nigeria Iceland 7549
Poland Colombia 7555
Croatia Nigeria 7529


Looping through matches:   9%|▉         | 6/64 [00:00<00:03, 16.06it/s]

Brazil Costa Rica 7548
Germany Mexico 7534
Australia Peru 7562
Serbia Brazil 7565


Looping through matches:  19%|█▉        | 12/64 [00:00<00:02, 17.46it/s]

...

Colombia England 7585
Sweden Switzerland 7586
Croatia England 8656
Sweden England 8651


Looping through matches:  91%|█████████ | 58/64 [00:03<00:00, 18.46it/s]

South Korea Germany 7567
Colombia Japan 7541
France Belgium 8655
Belgium England 8657


Looping through matches: 100%|██████████| 64/64 [00:03<00:00, 18.62it/s]

Russia Egypt 7540
Germany Sweden 7551
Serbia Switzerland 7550
Tunisia England 7537


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
groupedtime.to_parquet('/content/drive/MyDrive/Norwich/time.parquet',index=False)